In [1]:
from IPython.display import HTML
HTML(open('../../Python/style.css').read())

In [2]:
%load_ext nb_mypy

Version 1.0.5


# Taoistic Search

The holy book of Taoism, the <a href="https://en.wikipedia.org/wiki/Tao_Te_Ching">Tao Te Ching (道德經)</a>, contains the following wisdom:

<p>
<center style="color:blue; background-color:yellow; font-family: Brush Script MT, cursive; font-size:40px; padding: 12px">
    <a href="https://en.wikipedia.org/wiki/A_journey_of_a_thousand_miles_begins_with_a_single_step">
        A yourney of a thousand miles begins but with a single step.</a>
</center>
</p>

As written [here](https://www.linkedin.com/pulse/journey-thousand-miles-david-cheung/) this saying has also been used by Chairman
[Mao Zedong (毛泽东)](https://en.wikipedia.org/wiki/Mao_Zedong).

Taoistic search is based on the following principle: 
- Split the search problem into subproblems that can be readily solved.  
- Solve these problems one by one.
- Combine the solutions of the subproblems to a solution of the given *search problem*.

Hence, *taoistic search* really is a variation of the [divide-and-conquer](https://en.wikipedia.org/wiki/Divide-and-conquer_algorithm) paradigm.

We will use *taoistic search* to solve the search problem for the 15-puzzle where the states `Start` and `Goal` are defined
as follows.

In [3]:
Row   = tuple[str, ...]
State = tuple[Row, ...]

In [4]:
Start: State = ( (  '0', '14',  '8', '12' ),
                 ( '10', '11', '13',  '9' ),
                 (  '6',  '2',  '4', '15' ),
                 (  '3',  '5',  '7',  '1' )
               )

In [5]:
Goal: State  = ( (  '0',  '1',  '2',  '3' ),
                 (  '4',  '5',  '6',  '7' ),
                 (  '8',  '9', '10', '11' ),
                 ( '12', '13', '14', '15' )
               )

Please <span style="color:blue; background-color:yellow;">note</span> that we have defined the entries of these tuples as `str` and not as `int`.

In order to view these states more conveniently, we define a number of auxiliary functions in the following subsection. You need not concern yourself with these functions.  However, they must be executed for the animation to work.  

## Animation

In [6]:
import ipycanvas as cnv

The module `time` is part of the standard library so it is preinstalled.  We have imported it because we need the function `time.sleep(secs)` to pause the animation for a specified time.

In [7]:
import time

The global variable Colors specifies the colors of the tiles.

In [8]:
Colors = ['white', 'lightblue', 'pink', 'magenta', 'orange', 'red', 'yellow', 'lightgreen', 'gold',
          'CornFlowerBlue', 'Coral', 'Cyan', 'orchid', 'DarkSalmon', 'DeepPink', 'green'
         ] 

The global variable `gSize` specifies the size of one tile in pixels.

In [9]:
gSize = 100

The function `draw(State, canvas, dx, dy, tile, x)` draws a given `State` of the sliding puzzle, where `tile` has been moved by `offset` pixels into the direction `(dx, dy)`.

In [10]:
def draw(S: State, canvas: cnv.canvas, dx: int, dy: int, tile: str, offset: int) -> None:
    canvas.text_align    = 'center'
    canvas.text_baseline = 'middle'
    with cnv.hold_canvas(canvas):
        canvas.clear()
        n = len(S)
        for row in range(n):
            for col in range(n):
                tile_to_draw = S[row][col]
                if tile_to_draw != '*':
                    color = Colors[int(tile_to_draw)]
                else:
                    color = 'lightyellow'
                canvas.fill_style = color
                if tile_to_draw not in ('0', tile):
                    x = col * gSize
                    y = row * gSize
                    canvas.fill_rect(x, y, gSize, gSize)
                    canvas.stroke_rect(x, y, gSize, gSize)
                    canvas.line_width = 3.0
                    x += gSize // 2
                    y += gSize // 2
                    canvas.stroke_text(str(tile_to_draw), x, y)
                elif tile_to_draw == tile:
                    x = col * gSize + offset * dx
                    y = row * gSize + offset * dy
                    canvas.fill_rect(x, y, gSize, gSize)
                    canvas.stroke_rect(x, y, gSize, gSize)
                    canvas.line_width = 3.0
                    x += gSize // 2
                    y += gSize // 2
                    if tile_to_draw != 0:
                        canvas.stroke_text(str(tile_to_draw), x, y)

In [11]:
def create_canvas(n: int) -> cnv.canvas: 
    canvas = cnv.Canvas(size=(gSize * n, gSize * n))
    canvas.font = '60px serif'
    return canvas

In [12]:
def draw_state(S: State) -> None:
    n = len(S)
    canvas = create_canvas(n)
    draw(S, canvas, 0, 0, '+', 0)
    display(canvas) # type: ignore

The global variable `gDelay` controls the speed of the animation.  
<b style="color:red; background-color:lightyellow">If the animation is jerky on your computer, try increasing the value of the variable `gDelay` below.</b>

In [13]:
gDelay = 0.003

The function call `tile_and_direction(state, next_state)` takes a state and the state that follows this state and returns a triple (tile, dx, dy) where tile is the tile that is moved to transform `state` into `next_state` and `(dx, dy)` is the direction in which this tile is moved.

In [14]:
def find_tile(tile: str, S: State) -> tuple[int, int]:
    return None # type: ignore

In [15]:
def tile_and_direction(S: State, NS: State) -> tuple[str, int, int]:
    row0, col0 = find_tile('0', S)
    row1, col1 = find_tile('0', NS)
    return S[row1][col1], col0-col1, row0-row1

Given a list of states representing a solution to the sliding puzzle, the function call 
`animation(Solution)` animates the solution.

In [16]:
def animation(Solution: list[State]) -> None:
    start  = Solution[0]
    n      = len(start)
    canvas = create_canvas(n)
    draw(start, canvas, 0, 0, '0', 0)
    m = len(Solution)
    display(canvas) # type:ignore
    for i in range(m-1):
        state = Solution[i]
        tile, dx, dy = tile_and_direction(state, Solution[i+1])
        for offset in range(gSize+1):
            draw(state, canvas, dx, dy, tile, offset)
            time.sleep(gDelay)

## <a id="detailed_explanation">Taoistic Search: Detailed Explanation</a>

Let us begin by our explanation by drawing both the states `Start` and `Goal` of our *search problem*.

In [17]:
draw_state(Start)

Canvas()

In [18]:
draw_state(Goal)

Canvas()

To solve a specific case of the 15-puzzle, one effective strategy is to initially focus on positioning the tiles labeled `14` and `15` in their designated locations, without concerning ourselves with the placement of the other tiles. To facilitate this approach, we introduce two novel constructs, namely `Start1` and `Goal1`. These are referred to as <em style="color:blue">extended states</em>. In these <em style="color:blue">extended states</em>, all tiles irrelevant to the immediate goal of correctly positioning `14` and `15` are substituted with <em style="color:blue">wildcard tiles</em>, represented by the symbol `'*'`. <em style="color:blue">Extended states</em> are also commonly known as <em style="color:blue">patterns</em> in this context.


In [19]:
Start1 = ( ( '0', '14',  '*',  '*' ),
           ( '*',  '*',  '*',  '*' ),
           ( '*',  '*',  '*', '15' ),
           ( '*',  '*',  '*',  '*' )
         )
draw_state(Start1)

Canvas()

In [20]:
Goal1 = ( ( '*', '*',  '*',  '*' ),
          ( '*', '*',  '*',  '*' ),
          ( '*', '*',  '*',  '*' ),
          ( '*', '*', '14', '15' )
        )
draw_state(Goal1)

Canvas()


It is important to note that in the creation of the state `Goal1`, the tile labeled `'0'` has been substituted with a wildcard tile `'*'`, whereas this replacement does not occur in the state `Start1`. The reason is that we do 
not know where this tile will end up when we have reached the state `Goal1`.

The search problem, as defined by the states `Start1` and `Goal1`, is relatively straightforward to resolve. We record the sequence of <em style="color:blue">actions</em>—specifically, the movements of the empty tile—required to transform `Start1` into `Goal1`. These recorded actions are then applied to the state `Start`, leading to a potential outcome resembling the state `S1`, as depicted below.

In [21]:
S1 = ( ('10',  '8', '13', '12'),
       ('11',  '5',  '2',  '9'),
       ( '6',  '7',  '0',  '4'),
       ( '3',  '1', '14', '15')
     )
draw_state(S1)

Canvas()

Our next goal is to move the tiles numbered with `12` and `13` into their final position.  To this end we define the 
<em style="color:blue">extended states</em> `Start2` and `Goal2` as shown below.

In [22]:
Start2 = ( ( '*', '*', '13', '12' ),
           ( '*', '*',  '*',  '*' ),
           ( '*', '*',  '0',  '*' ),
           ( '*', '*', '14', '15' )
        )
draw_state(Start2)

Canvas()

In [23]:
Goal2 = ( ( '*',  '*',  '*',  '*'),
          ( '*',  '*',  '*',  '*'),
          ( '*',  '*',  '*',  '*'),
          ('12', '13', '14', '15')
        )
draw_state(Goal2)

Canvas()

Again, we solve the resulting search problem and remember the actions that transformed `Start2` into `Goal2`.  We apply these actions to the state `S1` and end up with `S1` being transformed into the state `S2` shown below.

In [24]:
S2 = ( ('10',  '5',  '8',  '9'),
       ( '6', '11',  '7',  '4'),
       ( '1',  '3',  '0',  '2'),
       ('12', '13', '14', '15')
     )
draw_state(S2)

Canvas()

Proceeding in this way we can solve the given instance of the 15-puzzle.  The solution that we find will not be optimal but it won't be too far from the optimal solution.

## Auxiliary Functions for the Sliding Puzzle

The function call `find_tile(tile, S)` finds the coordinates of the given `tile` in the state `S`.  The `tile` is represented as a string from the set 

`{'0', '1', ..., '15'}`,

where `'0'` represents the empty tile.  

<b style="color:blue; background-color:yellow">Nota bene:</b>
There are two types of **errors** that are commonly made in this exercise:
  - We represent the tiles as strings instead of numbers as we will later replace 
    some of these tiles by the wildcard character `'*'`.  If you use numbers instead
    of strings, your code will not work.
  - Do not mix up rows and colums.  Note that the columns correspond
    to the $x$-coordinate of a coordinate system, while the rows correspond to the $y$-coordinate.

In [25]:
def find_tile(tile: str, S: State) -> tuple[int, int]:
    n = len(S)
    for row in range(n):
        for col in range(n):
            if S[row][col] == tile:
                return row, col
    return None # type: ignore

Since A$^*$-search stores the set of states that have been visited, we have to represent states by immutable objects and hence we represent the states as tuples of tuples.  In order to be able to change these states, we have to transform these tuples of tuples into lists of lists.  The function `to_list` transforms a tuple of tuples into a list of lists.

In [26]:
def to_list(S: State) -> list[list[str]]:
    return [list(row) for row in S]

The function `to_tuple` transforms a list of lists into a tuple of tuples.

In [27]:
def to_tuple(S: list[list[str]]) -> State:
    return tuple(tuple(row) for row in S)

Given a `State` that satisfies 
```
    State[row][col] == '0'
```
and a direction `(dx, dy)` that is an element form the set 
$\bigl\{ (1, 0), (-1, 0), (0, 1), (0, -1) \bigr\}$,
the function `move_dir` moves the empty tile in the direction `(dx, dy)`.

In [28]:
def move_dir(S: State, row: int, col: int, dx: int, dy: int) -> State:
    S = to_list(S)
    S[row     ][col     ] = S[row + dy][col + dx]
    S[row + dy][col + dx] = '0'
    return to_tuple(S)

Given a `State` of the sliding puzzle, the function `next_states(State)` computes all those states that can be reached from `State` in one step.

In [29]:
def next_states(S: State) -> set[State]:
    n          = len(S)
    row, col   = find_tile('0', S)
    NewStates  = set()
    Directions = [ (1, 0), (-1, 0), (0, 1), (0, -1) ]
    for dx, dy in Directions:
        if row + dy in range(n) and col + dx in range(n):
            NewStates.add(move_dir(S, row, col, dx, dy))
    return NewStates

The function `matches(Pattern, S)` checks, whether the *pattern* `Pattern` matches the state `S`.  A *pattern* is like a state but instead of numbers,  some of the entries of the list of lists have the form `'*'`.  The idea is that the string `'*'` is a wildcard that matches anything.

<b style="color:blue; background-color:yellow">Note</b> that `S` can also be an *extended state*, i.e. `S` can contain the wildcard `'*'`. 

In [ ]:
def matches(Pattern: State, S: State) -> bool:
    "your code here"

The function `manhattan` implemented below takes as argument two *extended states* `S1` and `S2` 
possibly containing wildcards and computes the *Manhattan distance* between these 
extended states. Basically, the manhattan distance measure the number of moves that it would take to transform `S1` into `S2` if we were allowed to slide different tiles on top of each other.
When computing these distances,  tiles that are numbered with a wildcard are ignored.

In [ ]:
def manhattan(S1: State, S2: State) -> int:
    "your code here"

The function `find_numbers` takes a <em style="color:blue">pattern</em> `Pattern` as input and returns the
list of all tiles in `Pattern` that are labeled with a number.

In [ ]:
def find_numbers(Pattern: State) -> list[str]:
    Result = []
    n = len(Pattern)
    for row in range(n):
        for col in range(n):
            tile = Pattern[row][col]
            if tile != '*':
                Result.append(tile)
    return Result

The function `replace_numbers` takes two arguments:
- `Pattern` is an *extended state*,
- `Tiles` is a list of numbered tiles.

The state `Pattern` is transformed by replacing all tiles that are not a member of the list `Tiles`
with the wildcard character `*`.

In [ ]:
def replace_numbers(Pattern: State, Tiles: list[str]) -> State:
    "your code here"

The function `intermediate_goals` is called with two parameters:
- `Goal` is a state of the 15-puzzle,
- `TilesList` is a list of list of numbers.
   For example, `TilesList` could be the list 
   
   `[ [14, 15], [12, 13] ]`.
   
   This list would specify that we want to create two intermediate 
   goals.  
   - The first goal would only have the tiles numbered `14`and `15`,
     while all other tiles would be replaced by wildcards.
   - The second goal would only have the tiles numbered `12`, `13`, `14`,
     and `15`, while all other tiles would be replaced by wildcards.

The function returns the list of intermediate goals. 

In [ ]:
def intermediate_goals(Goal: State, TilesList: list[list[str]]) -> list[State]:
    "your code here"

Given two extended states $P_1$ and $P_2$ the function `extract_move` returns a pair `(dx, dy)` such that we have:
$$ (r, c) = \texttt{find_tile}(0, P_1) \rightarrow \texttt{move_dir}(P_1, r, c, dx, dy) = P_2
$$
Hence `extract_move`($P_1$, $P_2$) computes the action that is necessary to transform $P_1$ into $P_2$.  This action is encoded as a direction $(dx, dy)$.  This is the direction to move the empty tile in $P_1$ to reach $P_2$.

In [ ]:
Move = tuple[int, int]

In [ ]:
def extract_move(P1: State, P2: State) -> Move:
    "your code here"

Given a list of extended states `PatternList` of the form
$$ \texttt{PatternList} = [P_1, P_2, \cdots, P_n] $$
the function `extract_move_list` computes a list of actions $[a_1, a_2, \cdots, a_{n-1}]$ such that
applying action $(a_i)$ to state $P_i$ results in state $P_{i+1}$.  
The actions are pairs of the form `(dx, dy)` that specify how the empty tile is to be moved.

In [ ]:
def extract_move_list(PatternList: list[State]) -> list[Move]:
    "your code here"

Given the list of actions `MoveList` of the form $[a_1, a_2, \cdots, a_{n-1}]$, the function
`apply_move_list` takes a state `State` and applies these action to `State`one by one.  The list of all
states produced this way is returned.  This list starts with the given state `State`.

In [ ]:
def apply_move_list(S: State, MoveList: list[Move]) -> list[State]:
    "your code here"

The function `stateToString` is useful for debugging purposes to transform a given state into a string.

In [30]:
def stateToString(S: State) -> str:
    n      = len(S)
    indent = " " * 4;
    line   = indent + "+---" * n + "+\n"
    result = line
    for row in range(n):
        result += indent + "|"
        for col in range(n):
            cell = S[row][col]
            if isinstance(cell, str) and cell != '*':
                number = int(cell)
            if cell == "*":
                result += " * "
            elif number >= 10:
                result += str(cell) + " "
            elif 0 < number < 10:
                result += " " + cell + " "
            else: 
                result += "   "
            result += "|"
        result += "\n"
        result += line
    return result

In [31]:
print(stateToString(Start))

    +---+---+---+---+
    |   |14 | 8 |12 |
    +---+---+---+---+
    |10 |11 |13 | 9 |
    +---+---+---+---+
    | 6 | 2 | 4 |15 |
    +---+---+---+---+
    | 3 | 5 | 7 | 1 |
    +---+---+---+---+



## A$^*$ Search

In [32]:
import heapq

The function `search` takes three arguments to solve a *search problem*:
- `Start` is the start state of the search problem.
- `Goal`is the goal state.  This might be an *extended state*.
- `next_states` is a function with signature $\texttt{next_states}:Q \rightarrow 2^Q$, where $Q$ is the set of states.
  For every state $s \in Q$, $\texttt{next_states}(s)$ is the set of states that can be reached from $s$ in one step.
- `heuristic` is a function that takes two states as arguments.  It returns an estimate of the 
  length of the shortest path between these states.
If successful, `search` returns a path from `start` to `goal` that is a solution of the search problem
$$ \langle Q, \texttt{next_states}, \texttt{start}, \texttt{goal} \rangle. $$

Basically, the function `search` implements A$^*$ search, but instead of checking whether a state is identical to `Goal`, this function only tests whether a state *matches* goal.

The parameter `Goal` is not a state, but only a *pattern*.

In [33]:
from typing import Callable
NxtStFct  = Callable[[State], set[State]]
Heuristic = Callable[[State, State], int]

In [ ]:
def search(start: State, goal: State, next_states: NxtStFct, heuristic: Heuristic) -> list[State] | None:
    Visited: set[State] = set()
    PrioQueue = [ (heuristic(start, goal), [start]) ]
    while len(PrioQueue) > 0:
        _, Path = heapq.heappop(PrioQueue)
        state   = Path[-1]
        if state in Visited:
            continue
        if matches(goal, state):
            print(f'Number of states visited: {len(Visited)}')
            return Path
        for ns in next_states(state):           
            if ns not in Visited:
                prio = heuristic(ns, goal) + len(Path)
                heapq.heappush(PrioQueue, (prio, Path + [ns]))
        Visited.add(state)
    return None 

## Putting It All Together

Lets draw the start state and animate the solution that has been found.

In [ ]:
def main() -> list[State] | None:
    TilesList   = [['14', '15'], 
                   ['12', '13'], 
                   ['10', '11'], 
                   [ '8',  '9'], 
                   [ '3',  '7'], 
                   [ '2',  '6'], 
                   [ '4',  '5'],
                   [       '1']
                  ]
    PatternList = intermediate_goals(Goal, TilesList)
    State       = Start
    Solution    = []
    print('Start state:')
    draw_state(Start)
    for Pattern in PatternList:
        print('Trying to reach the following pattern:')
        draw_state(Pattern)
        Tiles = find_numbers(Pattern)
        ExtendedState = replace_numbers(State, Tiles + ['0'])
        Path = search(ExtendedState, Pattern, next_states, manhattan)
        if Path:
            MoveList = extract_move_list(Path)
            Path = apply_move_list(State, MoveList)
            print(f'The following state is reached after {len(Path)-1} steps:');
            State = Path[-1]
            draw_state(State)
            Solution += Path[:-1]
        else:
            return None
    Solution += [ Goal ]
    return Solution

In [ ]:
Path = main()
if Path:
    print(len(Path)-1)

In [ ]:
if Path:
    animation(Path) 

In [ ]:
def shorten(Solution: list[State]) -> list[State]:
    shorterSolution = []
    k = 0
    while k < len(Solution) - 1:
        shorterSolution.append(Solution[k])
        if k + 2 < len(Solution) and Solution[k] == Solution[k + 2]:
            k += 3
        else: 
            k += 1
    shorterSolution += [Solution[-1]]
    return shorterSolution

In [ ]:
if Path:
    animation(shorten(Path)) 

In [ ]:
if Path:
    print(len(shorten(Path))-1)